# Create toy population dataset

In [181]:
import numpy as np

In [182]:
pop_size = 12
str_len = 10
half_len = str_len//2

In [199]:
pop = np.array([np.random.permutation(np.arange(0, str_len)) for n in np.arange(pop_size)])
pop

array([[7, 1, 9, 0, 6, 3, 8, 5, 4, 2],
       [1, 2, 0, 3, 8, 5, 9, 4, 6, 7],
       [6, 5, 0, 9, 2, 8, 3, 4, 1, 7],
       [8, 2, 0, 9, 3, 1, 5, 4, 6, 7],
       [3, 9, 1, 4, 5, 6, 0, 7, 2, 8],
       [2, 8, 1, 0, 4, 7, 9, 5, 6, 3],
       [3, 9, 2, 4, 1, 5, 6, 8, 0, 7],
       [5, 1, 8, 2, 6, 7, 4, 0, 9, 3],
       [1, 3, 8, 7, 5, 0, 2, 9, 6, 4],
       [8, 4, 3, 5, 1, 7, 2, 9, 6, 0],
       [8, 2, 9, 4, 3, 6, 1, 7, 0, 5],
       [6, 5, 8, 3, 9, 7, 0, 1, 2, 4]])

# Sequence Selection on a single parent array

In [200]:
def vecArange(starts, delta):
    return np.repeat(np.expand_dims(starts, 1), delta, axis=1)+np.arange(0, delta)

In [201]:
seq_start = np.random.randint(0,half_len,pop_size)
seq_start

array([0, 3, 2, 3, 4, 3, 0, 3, 4, 2, 3, 0])

In [202]:
pop

array([[7, 1, 9, 0, 6, 3, 8, 5, 4, 2],
       [1, 2, 0, 3, 8, 5, 9, 4, 6, 7],
       [6, 5, 0, 9, 2, 8, 3, 4, 1, 7],
       [8, 2, 0, 9, 3, 1, 5, 4, 6, 7],
       [3, 9, 1, 4, 5, 6, 0, 7, 2, 8],
       [2, 8, 1, 0, 4, 7, 9, 5, 6, 3],
       [3, 9, 2, 4, 1, 5, 6, 8, 0, 7],
       [5, 1, 8, 2, 6, 7, 4, 0, 9, 3],
       [1, 3, 8, 7, 5, 0, 2, 9, 6, 4],
       [8, 4, 3, 5, 1, 7, 2, 9, 6, 0],
       [8, 2, 9, 4, 3, 6, 1, 7, 0, 5],
       [6, 5, 8, 3, 9, 7, 0, 1, 2, 4]])

In [203]:
seq = vecArange(seq_start,half_len)
seq

array([[0, 1, 2, 3, 4],
       [3, 4, 5, 6, 7],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8],
       [3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [0, 1, 2, 3, 4]])

In [206]:
subseq = pop[np.arange(pop.shape[0])[:, None], seq]
subseq

array([[7, 1, 9, 0, 6],
       [3, 8, 5, 9, 4],
       [0, 9, 2, 8, 3],
       [9, 3, 1, 5, 4],
       [5, 6, 0, 7, 2],
       [0, 4, 7, 9, 5],
       [3, 9, 2, 4, 1],
       [2, 6, 7, 4, 0],
       [5, 0, 2, 9, 6],
       [3, 5, 1, 7, 2],
       [4, 3, 6, 1, 7],
       [6, 5, 8, 3, 9]])

## Remove subsequences from other parents

In [262]:
pop_roll = np.roll(pop,1,axis=0)
pop_roll

array([[6, 5, 8, 3, 9, 7, 0, 1, 2, 4],
       [7, 1, 9, 0, 6, 3, 8, 5, 4, 2],
       [1, 2, 0, 3, 8, 5, 9, 4, 6, 7],
       [6, 5, 0, 9, 2, 8, 3, 4, 1, 7],
       [8, 2, 0, 9, 3, 1, 5, 4, 6, 7],
       [3, 9, 1, 4, 5, 6, 0, 7, 2, 8],
       [2, 8, 1, 0, 4, 7, 9, 5, 6, 3],
       [3, 9, 2, 4, 1, 5, 6, 8, 0, 7],
       [5, 1, 8, 2, 6, 7, 4, 0, 9, 3],
       [1, 3, 8, 7, 5, 0, 2, 9, 6, 4],
       [8, 4, 3, 5, 1, 7, 2, 9, 6, 0],
       [8, 2, 9, 4, 3, 6, 1, 7, 0, 5]])

In [263]:
for n, child in enumerate(subseq):
    print(pop_roll[n,np.isin(pop_roll,child,invert=True)])

IndexError: too many indices for array

# Sequence Selection with combined parent arrays

In [111]:
parents = np.concatenate((pop,pop_roll),axis=1).reshape(-1,2,str_len)
parents

array([[[0, 4, 2, 1, 6, 7, 8, 3, 5, 9],
        [6, 3, 0, 9, 5, 8, 1, 2, 4, 7]],

       [[9, 0, 1, 8, 4, 6, 7, 2, 5, 3],
        [0, 4, 2, 1, 6, 7, 8, 3, 5, 9]],

       [[6, 3, 1, 5, 8, 7, 4, 9, 2, 0],
        [9, 0, 1, 8, 4, 6, 7, 2, 5, 3]],

       [[8, 9, 4, 5, 3, 0, 7, 2, 6, 1],
        [6, 3, 1, 5, 8, 7, 4, 9, 2, 0]],

       [[8, 3, 7, 0, 1, 9, 6, 4, 2, 5],
        [8, 9, 4, 5, 3, 0, 7, 2, 6, 1]],

       [[6, 1, 3, 2, 5, 7, 8, 9, 4, 0],
        [8, 3, 7, 0, 1, 9, 6, 4, 2, 5]],

       [[6, 4, 8, 7, 2, 5, 0, 9, 1, 3],
        [6, 1, 3, 2, 5, 7, 8, 9, 4, 0]],

       [[4, 2, 1, 6, 7, 8, 5, 3, 9, 0],
        [6, 4, 8, 7, 2, 5, 0, 9, 1, 3]],

       [[9, 1, 7, 0, 4, 2, 3, 5, 8, 6],
        [4, 2, 1, 6, 7, 8, 5, 3, 9, 0]],

       [[3, 8, 5, 0, 7, 2, 9, 6, 1, 4],
        [9, 1, 7, 0, 4, 2, 3, 5, 8, 6]],

       [[1, 7, 4, 5, 6, 0, 9, 3, 8, 2],
        [3, 8, 5, 0, 7, 2, 9, 6, 1, 4]],

       [[6, 3, 0, 9, 5, 8, 1, 2, 4, 7],
        [1, 7, 4, 5, 6, 0, 9, 3, 8, 2]]])

In [153]:
parents

array([[[0, 4, 2, 1, 6, 7, 8, 3, 5, 9],
        [6, 3, 0, 9, 5, 8, 1, 2, 4, 7]],

       [[9, 0, 1, 8, 4, 6, 7, 2, 5, 3],
        [0, 4, 2, 1, 6, 7, 8, 3, 5, 9]],

       [[6, 3, 1, 5, 8, 7, 4, 9, 2, 0],
        [9, 0, 1, 8, 4, 6, 7, 2, 5, 3]],

       [[8, 9, 4, 5, 3, 0, 7, 2, 6, 1],
        [6, 3, 1, 5, 8, 7, 4, 9, 2, 0]],

       [[8, 3, 7, 0, 1, 9, 6, 4, 2, 5],
        [8, 9, 4, 5, 3, 0, 7, 2, 6, 1]],

       [[6, 1, 3, 2, 5, 7, 8, 9, 4, 0],
        [8, 3, 7, 0, 1, 9, 6, 4, 2, 5]],

       [[6, 4, 8, 7, 2, 5, 0, 9, 1, 3],
        [6, 1, 3, 2, 5, 7, 8, 9, 4, 0]],

       [[4, 2, 1, 6, 7, 8, 5, 3, 9, 0],
        [6, 4, 8, 7, 2, 5, 0, 9, 1, 3]],

       [[9, 1, 7, 0, 4, 2, 3, 5, 8, 6],
        [4, 2, 1, 6, 7, 8, 5, 3, 9, 0]],

       [[3, 8, 5, 0, 7, 2, 9, 6, 1, 4],
        [9, 1, 7, 0, 4, 2, 3, 5, 8, 6]],

       [[1, 7, 4, 5, 6, 0, 9, 3, 8, 2],
        [3, 8, 5, 0, 7, 2, 9, 6, 1, 4]],

       [[6, 3, 0, 9, 5, 8, 1, 2, 4, 7],
        [1, 7, 4, 5, 6, 0, 9, 3, 8, 2]]])

In [154]:
seq_start = np.random.randint(0,half_len,pop_size).repeat(2) #repeat is so pairs of parents have same subseq
seq_start

array([3, 3, 4, 4, 2, 2, 3, 3, 4, 4, 3, 3, 3, 3, 0, 0, 2, 2, 4, 4, 2, 2,
       0, 0])

In [172]:
parent_seq = vecArange(seq_start,half_len).reshape(-1,2,half_len)
parent_seq

array([[[3, 4, 5, 6, 7],
        [3, 4, 5, 6, 7]],

       [[4, 5, 6, 7, 8],
        [4, 5, 6, 7, 8]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[3, 4, 5, 6, 7],
        [3, 4, 5, 6, 7]],

       [[4, 5, 6, 7, 8],
        [4, 5, 6, 7, 8]],

       [[3, 4, 5, 6, 7],
        [3, 4, 5, 6, 7]],

       [[3, 4, 5, 6, 7],
        [3, 4, 5, 6, 7]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[4, 5, 6, 7, 8],
        [4, 5, 6, 7, 8]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]])

In [196]:
parents[np.arange(parents.shape[0])[:, None], parent_seq]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (12,1) (12,2,5) 